<div style="
  padding: 30px;
  text-align: center;" class='row'>
<div style="float:left;width: 15%;" class='column'><a href="https://www.colombiacompra.gov.co"><img alt="Logo Colombia Compra Eficiente" id="logocce" src="https://www.colombiacompra.gov.co/sites/cce_public/files/Images/logocce.png" style="height: 45px;"></a></div>
    <div style="float:left;width: 70%;" class='column'>
        <h1>Carga de bases datos abiertos
        </h1> 
    </div>
<div style="float:left;width: 15%;" class='column'><a href="https://www.dnp.gov.co/" target="_blank"><img class="float-right" id="logodnp" src="https://www.colombiacompra.gov.co/sites/cce_public/files/logo_dnp.jpg" style="width: 200px;"></a></div>
</div>

## 1. IDENTIFICACIÓN DEL INSUMO

|||
|:--|:--|
|**Fecha**|Mayo 2022|
|**Ciudad**|Bogotá D.C.|
|**Esquema de presentación del insumo**|Cuaderno Jupyter|
|**Título del insumo**| **Consideraciones para la consulta en datos abiertos**|
|**Descripción y alcance**|Script para la consulta de bases utilizando datos abiertos.|
|**Periodicidad del insumo**|único|
|**Solicitante**|No aplica|
|**Versión del insumo**|Final|

## 2. DESTINO Y AUTORES DEL INFORME / INSUMO

|||
|:--|:--|
|**Destinatario**|<table align='left'><tr><td>*Nombre:*</td> <td>Equipo analítica EMAE</td></tr> <tr><td>*Cargo:*</td> <td>NA</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE</td></tr></table>|
|**Autores**|<table><tr><td>*Nombre:*</td> <td>Carlos Isaac Zainea Maya</td></tr> <tr><td>*Cargo:*</td> <td>Contratista</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|
|**Aprobación**|<table><tr><td>*Nombre:*</td> <td>Catalina Pimienta</td></tr> <tr><td>*Cargo:*</td> <td>Subdirectora Estudios de Mercado y Abastecimiento Estratégico</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|

# Reportes interactivos

En este apartado crearemos un reporte interactivo usando la herramienta [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/), en conjunto con [Google Colaboratory](https://colab.research.google.com/) el usuario final puede capacitar a personal no tecnico para hacer consultas simples de datos abiertos.

Iniciemos cargando paquetes:

In [1]:
import pandas as pd
from sodapy import Socrata
import datetime

client = Socrata('www.datos.gov.co',None)

Como vamos a utilizar diferentes fuentes de datos, los siguientes diccionarios de Python, codifican cada base con su respectivo identificador del portal de datos abiertos. Observe que la sintáxis aquí utilizada solo esta mostrando el identificador y relaciona cada identificador con el nombre correspondiente. Si se pretende generar una herramienta más grande deben incluirse diccionarios o rutinas de integración de información. 

In [2]:
variables_dict = {"f789-7hwg": ["uid","nom_razon_social_contratista","identificacion_del_contratista","nombre_del_represen_legal","identific_representante_legal","fecha_de_firma_del_contrato"],
                 "jbjy-vk9h": ["id_contrato" ,"proveedor_adjudicado","documento_proveedor","nombre_representante_legal","identificaci_n_representante_legal","fecha_de_firma"],
                 "rgxm-mmea": ["identificador_de_la_orden ","proveedor","nit_proveedor","fecha"]}
Identificadores={"SECOP I":"f789-7hwg","SECOP II":"jbjy-vk9h","TVEC":"rgxm-mmea"}

Fechas={"f789-7hwg":"fecha_de_firma_del_contrato","jbjy-vk9h": "fecha_de_firma","rgxm-mmea":"fecha"}
Rep={"f789-7hwg":"nombre_del_represen_legal","jbjy-vk9h": "nombre_representante_legal","rgxm-mmea":"proveedor"}


Las funciones de ipywidgets que utilizaremos son:

`interact`: Genera un modulo interactivo según una función previamente definida.

`interactive`: Genera un modulo interactivo según una función previamente definida y permite guardarlo en una variable.

`DatePicker`: Un widget para escoger fechas.


In [3]:
from ipywidgets import interact, interactive
import ipywidgets as widgets

In [4]:
Fechainicio = widgets.DatePicker(
    description='Fecha Inicio',
    disabled=False,
    value = datetime.date(2022, 5, 1)
)
Fechafin = widgets.DatePicker(
    description='Fecha Inicio',
    disabled=False,
    value = datetime.date(2022, 6, 1)
)

La función a continuación permite hacer una consulta simple teniendo en cuenta las variables que se guardaron en el diccionario de variables `variables_dict`

In [5]:
variables_dict

{'f789-7hwg': ['uid',
  'nom_razon_social_contratista',
  'identificacion_del_contratista',
  'nombre_del_represen_legal',
  'identific_representante_legal',
  'fecha_de_firma_del_contrato'],
 'jbjy-vk9h': ['id_contrato',
  'proveedor_adjudicado',
  'documento_proveedor',
  'nombre_representante_legal',
  'identificaci_n_representante_legal',
  'fecha_de_firma'],
 'rgxm-mmea': ['identificador_de_la_orden ',
  'proveedor',
  'nit_proveedor',
  'fecha']}

Al final, usando interact e igualando los argumentos a los widgets de fecha previamente creados, podemos generar una herramienta interactiva que consulta estas variables en un intervalo de fechas dado. Adicionalmente, dejamos la variable texto que permite que podamos filtrar la consulta con el nombre del representante legal del proveedor:

In [6]:
def consultasimple(data_identifier,fechaini,fechafin,Representante_legal):   
    fechaini=fechaini.strftime("%Y-%m-%d")
    fechafin=fechafin.strftime("%Y-%m-%d")
    socrata_dataset_identifier = data_identifier
    lista_var=variables_dict[data_identifier]
    Query = """
    select """+", ".join(lista_var)+ """
    where
       """+Fechas[data_identifier]+'>="' + fechaini+'"'+""" and
       """+Fechas[data_identifier]+'<="' + fechafin+'"'+""" and
       LOWER(""" +Rep[data_identifier]+") like '%" +Representante_legal.lower()+"%'"+"""
    """+"""    
    limit
    1000
    """

    contratos_2022 = client.get(socrata_dataset_identifier, content_type="json", query=Query)

    secopII_2022_DF = pd.DataFrame(pd.DataFrame.from_dict(contratos_2022))
    return secopII_2022_DF

interact(consultasimple, data_identifier=Identificadores,fechaini=Fechainicio,fechafin=Fechafin,Representante_legal='Rep_legal')

interactive(children=(Dropdown(description='data_identifier', options={'SECOP I': 'f789-7hwg', 'SECOP II': 'jb…

<function __main__.consultasimple(data_identifier, fechaini, fechafin, Representante_legal)>

La anterior es una herramienta simple que permite identificar los contratos suscritos o por suscribir a partir del nombre del representante legal de un proveedor dado.

## Aprobación
|||
|--|--|
|**Nombre**|Wilson Camilo Sanchez|
|**Cargo**|Gestor T11|
|**Área**|Subdirección EMAE|

![version](images/version.png)